In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

# funtions.count()
from pyspark.sql.functions import count

import pandas as pd

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [95.2 kB]
Hit:13 http:

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-19 19:07:24--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-10-19 19:07:25 (6.42 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
#Reading csv file from Amazon_review/vine_table
df = pd.read_csv('vine_table.csv')

In [7]:
df

,Unnamed: 0,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,0,RTIS3L2M1F5SM,5,0,0,N,Y
1,1,R1ZV7R40OLHKD,5,0,0,N,Y
2,2,R3BH071QLH8QMC,1,0,1,N,Y
3,3,R127K9NTSXA2YH,3,0,0,N,Y
4,4,R32ZWUXDJPW27Q,4,0,0,N,Y
...,...,...,...,...,...,...,...
1785992,1785992,RPC430LWZJ60T,5,1,1,N,N
1785993,1785993,R347MZT5FH6HRJ,5,2,2,N,N
1785994,1785994,RJ5BETZP0VIUS,1,0,2,N,N
1785995,1785995,R85QTDO2KZMGO,1,1,3,N,N


In [8]:
total_votes_df=df.loc[df["total_votes"]>=20]

In [9]:
total_votes_df.head()

,Unnamed: 0,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,55,R4PKAZRQJJX14,1,21,34,N,N
74,74,R2CI0Y288CC7E2,1,21,35,N,Y
209,209,R127WEQY2FM1T3,1,147,175,N,Y
289,289,R3EZ0EPYLDA34S,1,14,31,N,Y
483,483,R2FJ94555FZH32,2,55,60,N,N


In [10]:
helpful_votes_df=total_votes_df.loc[total_votes_df["helpful_votes"]/total_votes_df["total_votes"]>=0.5]

In [11]:
helpful_votes_df.head()

,Unnamed: 0,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,55,R4PKAZRQJJX14,1,21,34,N,N
74,74,R2CI0Y288CC7E2,1,21,35,N,Y
209,209,R127WEQY2FM1T3,1,147,175,N,Y
483,483,R2FJ94555FZH32,2,55,60,N,N
537,537,R1U3AR67RE273L,1,51,65,N,Y


In [12]:
total_helpful_votes=helpful_votes_df['star_rating'].count()
total_helpful_votes

40565

In [13]:
paid_reviews_df=helpful_votes_df[helpful_votes_df["vine"]=="Y"]

In [14]:
paid_reviews_df.head()

,Unnamed: 0,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
32611,32611,R3KKUSGFZWSUIY,5,56,63,Y,N
33112,33112,R10FO5UKKVZBK2,3,23,23,Y,N
69680,69680,RM4KSGEOR7MU1,5,19,24,Y,N
155361,155361,RG7VRMYLEXD23,4,22,26,Y,N
239327,239327,R11O4YSCPSNL6L,3,20,26,Y,N


In [15]:
unpaid_reviews_df=helpful_votes_df[helpful_votes_df["vine"]=="N"]

In [16]:
unpaid_reviews_df.head()

,Unnamed: 0,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,55,R4PKAZRQJJX14,1,21,34,N,N
74,74,R2CI0Y288CC7E2,1,21,35,N,Y
209,209,R127WEQY2FM1T3,1,147,175,N,Y
483,483,R2FJ94555FZH32,2,55,60,N,N
537,537,R1U3AR67RE273L,1,51,65,N,Y


In [17]:
star5_review_paid_df=paid_reviews_df[paid_reviews_df['star_rating']==5]
star5_review_paid_df.head()

,Unnamed: 0,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
32611,32611,R3KKUSGFZWSUIY,5,56,63,Y,N
69680,69680,RM4KSGEOR7MU1,5,19,24,Y,N
335897,335897,R286MFBAJ8NPD6,5,46,51,Y,N
364816,364816,R1JRR530H4COA2,5,22,28,Y,N
367440,367440,RQ5WD90PUNBU9,5,21,24,Y,N


In [18]:
total_paid_5stars= star5_review_paid_df['star_rating'].count()
total_paid_5stars

48

In [19]:
star5_review_unpaid_df=unpaid_reviews_df[unpaid_reviews_df['star_rating']==5]
star5_review_unpaid_df.head()

,Unnamed: 0,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
1402,1402,R36O341WWXXKNP,5,28,31,N,N
2507,2507,R29BOS5HMAY1LO,5,88,110,N,N
3932,3932,RDX2ZZ46AM343,5,32,34,N,Y
4133,4133,R60GI4Z1CNGGV,5,17,20,N,Y
5278,5278,R2FAARI3JQO9XQ,5,29,30,N,Y


In [20]:
total_unpaid_5stars= star5_review_unpaid_df['star_rating'].count()
total_unpaid_5stars

15663

In [23]:
percentage_paid_5star_reviews =(total_paid_5stars/total_helpful_votes*100)


In [25]:
percentage_paid_5star_reviews

0.11832860840626155

In [24]:
percentage_unpaid_5star_reviews=(total_unpaid_5stars/total_helpful_votes*100)

In [26]:
percentage_unpaid_5star_reviews

38.612104030568226